In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 5       # number of variables
num_ineq = 5      # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_exact_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:24<00:00,  4.90it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     -0.022894               0.0      0.203028
std       0.154194               0.0      0.108530
min      -0.182410               0.0      0.075976
25%      -0.138515               0.0      0.122437
50%      -0.078114               0.0      0.177850
75%       0.034901               0.0      0.248992
max       0.678077               0.0      1.009168
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_rnd_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [12:26<00:00,  1.34it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.043882          0.040585      0.740334
std       0.137256          0.048872      1.877338
min      -0.182410          0.000000      0.149239
25%      -0.144368          0.000000      0.447507
50%      -0.090895          0.020409      0.690370
75%       0.002064          0.071587      0.802516
max       0.552763          0.265027     59.526771
Number of infeasible solution: 622


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_n1_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:31<00:00, 10.98it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.110827               0.0      0.090344
std       1.782178               0.0      0.033578
min      -0.084455               0.0      0.059285
25%       0.112639               0.0      0.076307
50%       0.299847               0.0      0.077373
75%       1.149328               0.0      0.079154
max       7.894967               0.0      0.233585
Number of infeasible solution: 0


### Heuristic - N3

In [12]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_n3_5-5.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:43<00:00,  9.68it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      0.226923               0.0      0.102537
std       0.414379               0.0      0.045570
min      -0.180625               0.0      0.073249
25%      -0.021205               0.0      0.076886
50%       0.069121               0.0      0.078802
75%       0.264289               0.0      0.107023
max       2.196364               0.0      0.342980
Number of infeasible solution: 0


## Learnable Rounding

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 132.74
Epoch 1, Validation Loss: 12.75
Epoch 2, Validation Loss: 1.24
Epoch 3, Validation Loss: 1.05
Epoch 4, Validation Loss: 0.98
Epoch 5, Validation Loss: 0.91
Epoch 6, Validation Loss: 0.87
Epoch 7, Validation Loss: 0.80
Epoch 8, Validation Loss: 0.82
Epoch 9, Validation Loss: 0.73
Epoch 10, Validation Loss: 0.68
Epoch 11, Validation Loss: 0.69
Epoch 12, Validation Loss: 0.62
Epoch 13, Validation Loss: 0.66
Epoch 14, Validation Loss: 0.58
Epoch 15, Validation Loss: 0.55
Epoch 16, Validation Loss: 0.50
Epoch 17, Validation Loss: 0.53
Epoch 18, Validation Loss: 0.56
Epoch 19, Validation Loss: 0.47
Epoch 20, Validation Loss: 0.44
Epoch 21, Validation Loss: 0.46
Epoch 22, Validation Loss: 0.45
Epoch 23, Validation Loss: 0.52
Epoch 24, Validation Loss: 0.40
Epoch 25, Validation Loss: 0.40
Epoch 26, Validation Loss: 0.41
Epoch 27, Validation Loss: 0.43
Epoch 28, Validation Loss: 0.42
Epoch 29, Validation Loss: 0.41
Epoch 30, Validation Loss: 0.52
Epoch 31, Valid

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_5-5.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 247.55it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.202430          0.000038      0.002979
std       0.196878          0.000889      0.001608
min      -0.042898          0.000000      0.000999
25%       0.041265          0.000000      0.002005
50%       0.163571          0.000000      0.002512
75%       0.278137          0.000000      0.003121
max       1.064301          0.025181      0.014828
Number of infeasible solution: 2


## Learnable Threshold

In [19]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [20]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [21]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [22]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 126.15
Epoch 1, Validation Loss: 17.68
Epoch 2, Validation Loss: 1.36
Epoch 3, Validation Loss: 1.03
Epoch 4, Validation Loss: 0.97
Epoch 5, Validation Loss: 0.91
Epoch 6, Validation Loss: 0.84
Epoch 7, Validation Loss: 0.80
Epoch 8, Validation Loss: 0.78
Epoch 9, Validation Loss: 0.80
Epoch 10, Validation Loss: 0.69
Epoch 11, Validation Loss: 0.69
Epoch 12, Validation Loss: 0.69
Epoch 13, Validation Loss: 0.67
Epoch 14, Validation Loss: 0.60
Epoch 15, Validation Loss: 0.63
Epoch 16, Validation Loss: 0.61
Epoch 17, Validation Loss: 0.58
Epoch 18, Validation Loss: 0.58
Epoch 19, Validation Loss: 0.54
Epoch 20, Validation Loss: 0.58
Epoch 21, Validation Loss: 0.67
Epoch 22, Validation Loss: 0.56
Epoch 23, Validation Loss: 0.61
Epoch 24, Validation Loss: 0.54
Epoch 25, Validation Loss: 0.57
Epoch 26, Validation Loss: 0.54
Epoch 27, Validation Loss: 0.55
Epoch 28, Validation Loss: 0.52
Epoch 29, Validation Loss: 0.50
Epoch 30, Validation Loss: 0.48
Epoch 31, Valid

In [23]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_5-5.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 264.66it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.215118          0.000171      0.002860
std       0.173971          0.002777      0.001468
min       0.091262          0.000000      0.000999
25%       0.100285          0.000000      0.002001
50%       0.131750          0.000000      0.002510
75%       0.275424          0.000000      0.003034
max       1.052636          0.075770      0.014105
Number of infeasible solution: 6
